# Recording a Potential Energy Profile: The Intrinsic Reaction Coordinate (IRC)

A transition state search merely guarantees that one reaches a
first-order saddle point on the PES; there is, however, no guarantee
that it is really linked to the reactant and product. (For instance,
there may be further transition states lying in between.) Although the
visualisation of the imaginary vibrational mode may give some hint as to
whether one has found a reasonable saddle point, this information is not
sufficient to guarantee a connection to the reactant and product well.
Following the negative Hessian eigenvalues from the transition state to
the next local minimum, on the other hand, will immediately reveal
whether such a connection exists. This Hessian-following in
mass-weighted coordinates is referred to as a search along the
*Intrinsic Reaction Coordinate* (IRC). This search will create a
potential energy profile for the reaction, as well as a trajectory that
visualises the lowest energy path from reactant to TS to product or reactant (depending on the direction that one chooses). 

As psi4 automatically performs a frequency calculation we can verify whether the transition state that was computed is the proper transition state. 

In [1]:
import psi4
import numpy as np

import sys
sys.path.append("..")
from helpers import *

In [ ]:
# Read the transition state coordinates from disk
with open('ts.xyz', 'r') as f:
    ts_string = f.read()  # this is the xyz file as a string
    print(ts_string)

In [74]:
ts = psi4.geometry(f"""
-1 1
symmetry c1
#coordinates here""")

# Choose a direction

Start from the TS that we optimized and run the IRC calculation in one direction.

You can choose `forward` or `backward` for the `irc_direction`



In [ ]:
irc_direction =  'forward' # 'backward'

In [8]:
psi4.core.set_output_file(f'irc.log', False)
psi4.set_options({
     "geom_maxiter":500,
     "full_hess_every":0,
     "opt_type":"irc",
     "irc_step_size":0.25,
     "ensure_bt_convergence":True,
     "irc_direction":irc_direction,
     })

E, history = psi4.optimize('b3pw91/6-31+G*',molecule=ts, return_history=True)

 55 displacements needed.
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55
Optimizer: Optimization complete!


In [67]:
coordinates = history['coordinates']
energies = history['energy']


# Convert to kcal mol-1
e = [] # use a list comprehension
# set highest value to 0
e = [] # convert list to np.array and use broadcasting to substract max value, you can use np.max()

## Pickling the result. 
We write the resulting energies and coordiations to a pickle file which is a binary representation of python objects on disk. 

We can load this in another notebook or you can share it with a colleague (see below).

In [34]:
import pickle
import json

In [68]:
pickle.dump((backward_e, 
             mol2traj(backward, coordinates_backward), 
             coord2traj_array(coordinates_backward)), open( f"{irc_direction}.p", "wb" ) )

Calculating the IRC takes some time. We therefore ask you to only calculate one part of the trajectory and paired you with one of you colleagues so that you can still look at the full trajectory. See Moodle for the list of partners. 

## Upload the part of the IRC that you calculated to share with your colleague

We use `curl` which is a linux package to perform http requests. We use the free `file.io` service that allows you share file anonymously without registering. 
The file can only be downloaded once. You can of course also use any other method to share the pickled files. 

In [ ]:
# Upload, # change the name of the file forward.p
response = !curl -F "file=@forward.p" https://file.io

try:
    print("Share this code with you colleague: ", json.loads(response[6])['key'])
except:
    print('Something went wrong, check the name of the file and check the response')
    print(response)